<a href="https://colab.research.google.com/github/henrilhos/genetic-algorithm/blob/main/knapsack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema da mochila

## Alunos

- Henrique de Castilhos
- Vinícius Ferri

### O Problema

Você ficará isolado na natureza selvagem. 

A única coisa que você poderá levar é uma mochila que suporta no máximo 30 kg. 
# New Section
Você possui diversos itens de sobrevivência, cada um possui "pontos de sobrevivência" (dados para cada item de acordo com a tabela). 

Seu objetivo é maximizar os pontos de sobrevivência.


| Item           | Peso | Pontos |
|----------------|------|--------|
| Saco de dormir | 15   | 15     |
| Corda          | 3    | 10     |
| Canivete       | 2    | 10     |
| Tocha          | 5    | 5      |
| Garrafa        | 9    | 8      |
| Comida         | 20   | 17     |

In [1]:
import sys
import numpy as np

Cálculo do `fitness` de cada solução na população atual.

A função `cal_pop_fitness` calcula a soma dos produtos entre cada entrada e seu peso correspondente.

In [14]:
def cal_pop_fitness(points, weights, pop):
  fitness_points = np.sum(pop * points, axis=1)
  fitness_weights = np.sum(pop * weights, axis=1)

  fitness_points[fitness_weights > 30] = -sys.maxsize - 1
  
  return fitness_points

Seleciona os melhores indivíduos na geração atual para serem pais para o cruzamento.

In [3]:
def select_mating_pool(pop, fitness, num_parents):
  parents = np.empty((num_parents, pop.shape[1]))

  for parent_num in range(num_parents):
    max_fitness_idx = np.where(fitness == np.max(fitness))
    max_fitness_idx = max_fitness_idx[0][0]
    parents[parent_num, :] = pop[max_fitness_idx, :]
    fitness[max_fitness_idx] = -sys.maxsize - 1
  
  return parents

`crossover_point` é onde o cruzamento acontece entre os dois genitores. Geramos um número aleatório entre 1 e o tamanho do cromossomo.

In [4]:
def crossover(parents, offspring_size):
  offspring = np.empty(offspring_size)
  crossover_point = np.random.randint(1, offspring_size[1])

  for k in range(offspring_size[0]):
    # índice do primeiro genitor
    parent1_idx = k % parents.shape[0]
    # índice do segundo genitor
    parent2_idx = (k + 1) % parents.shape[0]
    # o novo filho terá a primeira parte de seus genes
    # oriunda do primeiro genitor
    offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
    # o novo filho terá a segunda parte de seus genes
    # oriunda do segundo genitor
    offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
  
  return offspring

A mutação transforma um gene único em cada filho, aleatoriamente. A mutação só ocorrerá se dentro da `mutation_rate` (por padrão 30%).

In [5]:
def mutation(offspring_crossover, mutation_rate=0.3):
  for idx in range(offspring_crossover.shape[0]):
    if np.random.random() < mutation_rate:
      # valor aleatório a ser adicionado
      random_idx = np.random.randint(0, offspring_crossover.shape[1])
      # random_value = np.random.uniform(-1.0, 1.0, 1)
      offspring_crossover[idx, random_idx] = offspring_crossover[idx, random_idx] - 1
  
  return offspring_crossover

In [18]:
# entradas da equação
points = [15, 10, 10, 5, 8, 17]
weights = [15, 3, 2, 5, 9, 20]
# número de pesos a otimizar
num_weights = 6

In [23]:
sol_per_pop = 10

# população tem 'sol_per_pop' cromossomos com 'num_weights' genes
pop_size = (sol_per_pop, num_weights)

# população inicial
new_population = np.random.randint(2, size=pop_size)

In [24]:
# algoritmo genético
num_generation = 100
num_parents_mating = 4

for generation in range(num_generation):
  print(f"Geração: {generation}")

  # medir o 'fitness' de cada cromossomo na população
  fitness = cal_pop_fitness(points, weights, new_population)

  print("Valores de fitness:")
  print(fitness)

  # selecionar os melhores pais na população para o cruzamento
  parents = select_mating_pool(new_population, fitness, num_parents_mating)

  print("Genitores selecionados:")
  print(parents)

  # formar a próxima geração usando crossover
  offspring_crossover = crossover(parents, offspring_size=(
      pop_size[0] - parents.shape[0], num_weights
  ))
  print("Resultado do crossover:")
  print(offspring_crossover)

  # adicionar variações aos filhos usando mutação
  offspring_mutation = mutation(offspring_crossover)
  print("Resultado da mutação:")
  print(offspring_mutation)

  # criar a nova população baseada nos pais e filhos
  new_population[0:parents.shape[0], :] = parents
  new_population[parents.shape[0]:, :] = offspring_mutation

  best_result = np.max(np.sum(new_population * points, axis=1))
  print(f"Melhor resultado depois da geração {generation}: {best_result}")

Geração: 0
Valores de fitness:
[-9223372036854775808                   18                   10
                   18 -9223372036854775808                   15
 -9223372036854775808 -9223372036854775808                   32
                    0]
Genitores selecionados:
[[0. 0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]]
Resultado do crossover:
[[0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 1. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]
Resultado da mutação:
[[0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]
Melhor resultado depois da geração 0: 42
Geração: 1
Valores de fitness:
[32 18 18 15 18  8 10 42 18  8]
Genitores selecionados:
[[0. 1. 1. 1. 0. 1.]
 [0. 0. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 0.]]
Resultado do crossover:
[[0. 1. 1. 1. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 1. 1. 0. 1.]
 [0. 1. 1. 1. 0. 1.]
 [0

In [22]:
fitness = cal_pop_fitness(points, weights, new_population)
best_match_idx = np.where(fitness == np.max(fitness))

print("Melhor solução: ", new_population[best_match_idx, :])
print("Fitness da melhor solução: ", fitness[best_match_idx])

Melhor solução:  [[[1 0 0 1 1 0]
  [1 0 0 1 1 0]
  [1 0 0 1 1 0]
  [1 0 0 1 1 0]
  [1 0 0 1 1 0]
  [1 0 0 1 1 0]
  [1 0 0 1 1 0]]]
Fitness da melhor solução:  [28 28 28 28 28 28 28]
